## Code to Chapter 3 of LangChain for Life Science and Healthcare book, by Dr. Ivan Reznikov

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/19Jz1nELzpxwBH1OoDnLbAQZakD33jsFx?usp=sharing)

This notebook demonstrates the core components of LangChain framework, specifically focusing on applications in life sciences and healthcare. We'll explore prompt templates, output parsers, chains, and runnable components that form the building blocks of LangChain applications.

## Package Installation and Setup

In [ ]:
#install all proper packages
#!pip install -qU langchain langchain-community langchain-core langchain-openai openai tiktoken chromadb pandas pypdf xmltodict
!pip install -qU langchain langchain_huggingface langchain-community langchain-core langchain-openai langchain-text-splitters \
  openai tiktoken chromadb pypdf xmltodict transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.0/755.0 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 106.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 58.7 MB/s eta

In [ ]:
!pip freeze | grep "lang\|openai\|tiktoken|\chroma|\transformers"

google-ai-generativelanguage==0.6.15
google-cloud-language==2.17.2
langchain==0.3.26
langchain-community==0.3.26
langchain-core==0.3.66
langchain-huggingface==0.3.0
langchain-openai==0.3.27
langchain-text-splitters==0.3.8
langcodes==3.5.0
langsmith==0.4.1
language_data==1.3.0
libclang==18.1.1
openai==1.93.0


In [ ]:
from google.colab import userdata
import os


# Set OpenAI API key from Google Colab's user environment or default
def set_openai_api_key(default_key: str = "YOUR_API_KEY") -> None:
    """Set the OpenAI API key from Google Colab's user environment or use a default value."""

    os.environ["OPENAI_API_KEY"] = userdata.get("LC4LS_OPENAI_API_KEY") or default_key


try:
    set_openai_api_key()
except Exception as e:
    print(e)

## Structured Output Parsing with Pydantic

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List

In [ ]:
# Define the patient data structure
class PatientAssessment(BaseModel):
    diagnosis: str = Field(description="Primary medical diagnosis")
    pain_level: int = Field(description="Pain level on scale of 0-10")
    symptoms: List[str] = Field(description="List of reported symptoms")
    requires_hospitalization: bool = Field(
        description="Whether patient needs to be hospitalized"
    )


# Create the parser
parser = PydanticOutputParser(pydantic_object=PatientAssessment)

This demonstrates **structured output parsing** - a crucial feature for healthcare applications:

1. **Pydantic Model Definition**:
   - `PatientAssessment` class defines the expected structure of medical assessments
   - Each field has type hints and descriptions for clarity
   - `Field()` provides metadata that helps the LLM understand what's expected

2. **PydanticOutputParser**:
   - Automatically generates format instructions for the LLM
   - Parses the LLM's response into a structured Python object
   - Validates data types and structure

3. **Prompt Template**:
   - `{patient_info}` is the input variable for patient data
   - `{format_instructions}` is automatically populated by the parser
   - The parser tells the LLM exactly how to format its response

4. **Chain Construction**:
   - Uses the `|` operator to create a pipeline: prompt → LLM → parser
   - Each component transforms the data before passing to the next

5. **Type Safety**:
   - The result is a fully typed Python object
   - No need for manual JSON parsing or type conversion
   - Perfect for integration with healthcare systems that require structured data

This approach is essential in both life sciences and healthcare where data consistency and validation are critical.

In [ ]:
# Create the prompt template
template = """
Based on the following patient information, provide a medical assessment:
Patient Information: {patient_info}

{format_instructions}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["patient_info"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [ ]:
# Set up the LLM
llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

# Create the chain
chain = prompt | llm | parser

In [ ]:
# Run the chain
result = chain.invoke(
    {
        "patient_info": "45-year-old male presenting with chest pain, shortness of breath, and fever of 101°F for the past 2 days. History of hypertension."
    }
)

print(f"Diagnosis: {result.diagnosis}")
print(f"Pain Level: {result.pain_level} (Type: {type(result.pain_level)})")
print(f"Symptoms: {result.symptoms} (Type: {type(result.symptoms)})")
print(
    f"Requires Hospitalization: {result.requires_hospitalization} (Type: {type(result.requires_hospitalization)})"
)

Diagnosis: Possible acute coronary syndrome or pneumonia
Pain Level: 7 (Type: <class 'int'>)
Symptoms: ['chest pain', 'shortness of breath', 'fever of 101°F'] (Type: <class 'list'>)
Requires Hospitalization: True (Type: <class 'bool'>)


In [ ]:
result, dict(result)

(PatientAssessment(diagnosis='Possible acute coronary syndrome or pneumonia', pain_level=7, symptoms=['chest pain', 'shortness of breath', 'fever of 101°F'], requires_hospitalization=True),
 {'diagnosis': 'Possible acute coronary syndrome or pneumonia',
  'pain_level': 7,
  'symptoms': ['chest pain', 'shortness of breath', 'fever of 101°F'],
  'requires_hospitalization': True})

## Hugging Face Model Integration

In [ ]:
# Load the model and tokenizer
model_name = "GT4SD/multitask-text-and-chemistry-t5-base-augm"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create a pipeline with the model and tokenizer
pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    num_beams=5,
)

# Create a LangChain HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipe)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


This demonstrates **local model integration** using Hugging Face, which is crucial for healthcare applications with privacy concerns:

1. **Specialized Chemistry Model**:
   - `GT4SD/multitask-text-and-chemistry-t5-base-augm` is a T5 model trained on chemistry and text tasks
   - Perfect for pharmaceutical and chemical research applications
   - Runs completely offline once downloaded

2. **Pipeline Configuration**:
   - `temperature=0`: Ensures deterministic, reproducible outputs (critical in healthcare)
   - `max_length=512`: Limits response length
   - `num_beams=5`: Uses beam search for higher quality outputs

3. **LangChain Integration**:
   - `HuggingFacePipeline` wraps the Hugging Face pipeline
   - Makes it compatible with LangChain's chain architecture
   - Enables consistent interface across different model providers

4. **Prompt Engineering Comparison**:
   - **Basic prompt**: Minimal context, relies on model's training
   - **Prompt 1**: Adds role context ("as a chemist")
   - **Prompt 2**: More elaborate role-playing with professional context
   - Demonstrates how prompt engineering affects model behavior

5. **Chain Architecture**:
   - Each chain follows the same pattern: prompt → model → output parser
   - `StrOutputParser()` extracts the text from the model's response
   - Shows how different prompts can be easily tested with the same underlying model

This approach is valuable for healthcare organizations that need to keep sensitive data on-premises while still leveraging AI capabilities.

In [ ]:
# Define prompt templates
basic_prompt = PromptTemplate.from_template("{input_text}")
prompt1 = PromptTemplate.from_template(
    "Continue the following phrase as a chemist: {input_text}"
)
prompt2 = PromptTemplate.from_template(
    "You are a professional chemistry researcher.\nFinish the following sentence: {input_text}"
)

In [ ]:
# Create chains for each prompt
basic_chain = basic_prompt | llm | StrOutputParser()
prompt1_chain = prompt1 | llm | StrOutputParser()
prompt2_chain = prompt2 | llm | StrOutputParser()

In [ ]:
# The text to be continued
TEXT = "The formula of dihydrogen monoxide is"

# Run the chains
basic_result = basic_chain.invoke({"input_text": TEXT})
prompt1_result = prompt1_chain.invoke({"input_text": TEXT})
prompt2_result = prompt2_chain.invoke({"input_text": TEXT})

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
print("Basic prompt result:", basic_result)
print("Prompt 1 result:", prompt1_result)
print("Prompt 2 result:", prompt2_result)

Basic prompt result: [O-][Mn](=O)(=O)=O
Prompt 1 result: [O-][Mn](=O)(=O)=O.[O-][Mn](=O)(=O)=O.[O-][Mn](=O)(=O)=O
Prompt 2 result: The molecule is a dihydrogen monoxide. It is a conjugate base of a dihydrogen monoxide(2+). It is a conjugate acid of a dihydrogen monoxide(1-).


As you can see above, prompts matter!

## Few-Shot Learning for Medical Reasoning

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples = [
    {
        "question": "Is Penicillin effective against E. coli?",
        "answer": """
Are follow-up questions needed here: Yes.
Follow up: What class of antibiotics does Penicillin belong to?
Intermediate answer: Penicillin belongs to the beta-lactam class of antibiotics.
Follow up: What mechanism of action does Penicillin have?
Intermediate answer: Penicillin interferes with the synthesis of the bacterial
cell wall.
Follow up: Is E. coli resistant to beta-lactam antibiotics?
Intermediate answer: Many strains of E. coli have developed resistance to
beta-lactam antibiotics, including Penicillin.
So the final answer is: Penicillin is generally not effective against E. coli due
to resistance.
""",
    },
    {
        "question": "Do Aspirin and Ibuprofen have the same mechanism of action?",
        "answer": """
Are follow-up questions needed here: Yes.
Follow up: What is the mechanism of action of aspirin?
Intermediate Answer: Aspirin works by inhibiting the enzyme cyclooxygenase (COX),
which reduces the formation of prostaglandins and thromboxanes, leading to its
anti-inflammatory and anticoagulant effects.
Follow up: What is the mechanism of action of Ibuprofen?
Intermediate Answer: Ibuprofen also inhibits the cyclooxygenase (COX) enzyme,
reducing the production of prostaglandins.
So the final answer is: Yes, both Aspirin and Ibuprofen have the same mechanism of
action, which is the inhibition of the COX enzyme.
""",
    },
]

This demonstrates **few-shot learning** with **chain-of-thought reasoning** - essential for complex medical reasoning:

1. **Few-Shot Learning Concept**:
   - Provides the model with examples of the desired reasoning pattern
   - Teaches the model to break down complex medical questions systematically
   - More effective than zero-shot prompting for specialized domains

2. **Chain-of-Thought Reasoning**:
   - Each example shows step-by-step logical progression
   - "Follow up questions" break complex problems into manageable parts
   - "Intermediate answers" provide reasoning transparency
   - Critical for medical applications where reasoning must be auditable

3. **Medical Domain Examples**:
   - **Example 1**: Antibiotic resistance reasoning
   - **Example 2**: Drug mechanism comparison
   - Both examples demonstrate multi-step medical reasoning

4. **Template Structure**:
   - `example_prompt`: Formats each individual example
   - `FewShotPromptTemplate`: Combines all examples with the new question
   - `suffix`: Adds the new question in the same format

5. **Comparison Study**:
   - First call: Direct LLM invocation without examples
   - Second call: Using the few-shot chain
   - Shows how examples improve the quality and structure of medical reasoning

6. **Complex Medical Question**:
   - Cancer genetics question requires multi-layered understanding
   - Tests the model's ability to apply the learned reasoning pattern
   - Demonstrates how few-shot learning scales to complex scenarios

In [ ]:
# setting the prompt template to process the examples list
example_prompt = PromptTemplate(
    input_variables=["question", "answer"], template="Question: {question}\n{answer}"
)

# setting the prompt template to be used with the LLM
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

In [ ]:
TEXT = "How do genetic mutations in oncogenes and tumor suppressor genes interact to drive the progression of metastatic cancer?"

In [ ]:
llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

In [ ]:
llm.invoke(TEXT).content

"Genetic mutations in oncogenes and tumor suppressor genes play crucial roles in the development and progression of metastatic cancer. Their interactions can create a complex network of signaling pathways that drive tumorigenesis, invasion, and metastasis. Here’s how these mutations interact and contribute to cancer progression:\n\n### Oncogenes\nOncogenes are mutated forms of normal genes (proto-oncogenes) that promote cell growth and division. When mutated, they can lead to uncontrolled cell proliferation. Common mechanisms of oncogene activation include point mutations, gene amplifications, and chromosomal rearrangements. Examples of oncogenes include:\n\n- **KRAS**: Mutations in KRAS lead to continuous activation of signaling pathways that promote cell growth and survival.\n- **MYC**: Overexpression of MYC can drive cell proliferation and metabolism.\n\n### Tumor Suppressor Genes\nTumor suppressor genes normally function to inhibit cell growth, repair DNA damage, or induce apoptosi

In [ ]:
basic_chain = prompt | llm | StrOutputParser()
basic_chain.invoke({"input": TEXT})

'Are follow-up questions needed here: Yes.  \nFollow up: What are oncogenes?  \nIntermediate Answer: Oncogenes are mutated forms of normal genes (proto-oncogenes) that promote cell growth and division. When mutated, they can lead to uncontrolled cell proliferation.  \nFollow up: What are tumor suppressor genes?  \nIntermediate Answer: Tumor suppressor genes are genes that normally help regulate cell growth and division, repair DNA, and promote apoptosis (programmed cell death). When these genes are mutated or inactivated, they lose their ability to control cell growth.  \nFollow up: How do mutations in oncogenes contribute to cancer progression?  \nIntermediate Answer: Mutations in oncogenes can lead to the overactivation of signaling pathways that promote cell proliferation and survival, contributing to tumor growth and metastasis.  \nFollow up: How do mutations in tumor suppressor genes contribute to cancer progression?  \nIntermediate Answer: Mutations in tumor suppressor genes can 

## Runnable Components and Composition

In [ ]:
from langchain_core.runnables import RunnableLambda

# A RunnableSequence constructed using the `|` operator
sequence = RunnableLambda(lambda x: x + 1) | RunnableLambda(lambda x: x * 2)

This section introduces **LangChain's Runnable interface** - the foundation for building complex AI workflows:

1. **RunnableLambda**:
   - Wraps simple Python functions into LangChain-compatible components
   - Enables functional programming patterns in AI pipelines
   - Provides consistent interfaces for invoke, batch, and stream operations

2. **Sequential Composition**:
   - `|` operator chains operations left-to-right
   - Output of first operation becomes input of second
   - Similar to Unix pipes but for AI workflows

3. **Parallel Composition**:
   - Dictionary syntax creates parallel branches
   - Single input is processed by multiple functions simultaneously
   - Results are combined into a dictionary with named outputs

4. **Batch Processing**:
   - `batch()` method processes multiple inputs efficiently
   - Maintains the same transformation logic across all inputs
   - Essential for processing large datasets in healthcare applications

5. **Flexibility**:
   - Can combine sequential and parallel operations
   - Building blocks for complex medical data processing pipelines
   - Enables sophisticated workflows while maintaining readability

This pattern is foundational for building scalable healthcare AI systems where data needs to flow through multiple processing steps.

In [ ]:
sequence.invoke(1)  # 4

4

In [ ]:
sequence.batch([1, 2, 3])  # [4, 6, 8]

[4, 6, 8]

In [ ]:
# A sequence that contains a RunnableParallel constructed using a dict literal
sequence = RunnableLambda(lambda x: x + 1) | {
    "mul_2": RunnableLambda(lambda x: x * 2),
    "mul_5": RunnableLambda(lambda x: x * 5),
}

In [ ]:
sequence.invoke(1)

{'mul_2': 4, 'mul_5': 10}

In [ ]:
sequence.batch([1, 2, 3])  # [4, 6, 8]

[{'mul_2': 4, 'mul_5': 10},
 {'mul_2': 6, 'mul_5': 15},
 {'mul_2': 8, 'mul_5': 20}]

## Practical Example: Temperature Conversion with LangChain

In [ ]:
from langchain_core.runnables import RunnableLambda


# Define temperature conversion functions
def kelvin_to_celsius(kelvin_temp):
    """Convert Kelvin to Celsius"""
    return kelvin_temp - 273.15


def celsius_to_fahrenheit(celsius_temp):
    """Convert Celsius to Fahrenheit"""
    return (celsius_temp * 9 / 5) + 32


def format_temperature_result(temp_dict):
    """Format the temperature conversion results"""
    return {
        "original_kelvin": temp_dict["kelvin"],
        "celsius": round(temp_dict["celsius"], 2),
        "fahrenheit": round(temp_dict["fahrenheit"], 2),
        "formatted_output": f"{temp_dict['kelvin']}K = {round(temp_dict['celsius'], 2)}°C = {round(temp_dict['fahrenheit'], 2)}°F",
    }


# Create RunnableLambda components
kelvin_to_celsius_runnable = RunnableLambda(kelvin_to_celsius)
celsius_to_fahrenheit_runnable = RunnableLambda(celsius_to_fahrenheit)
format_result_runnable = RunnableLambda(format_temperature_result)

This comprehensive example demonstrates **practical application of LangChain Runnables** for temperature conversion with medical context:

1. **Function Definition**:
   - Pure Python functions for temperature conversions
   - Follows single responsibility principle
   - Easy to test and maintain

2. **RunnableLambda Wrapping**:
   - Converts Python functions into LangChain components
   - Enables composition with other LangChain components
   - Provides consistent interface (invoke, batch, stream)

3. **Sequential Processing**:
   - Demonstrates step-by-step transformation
   - Kelvin → Celsius → Fahrenheit pipeline
   - Maintains intermediate results for transparency

4. **Parallel Processing**:
   - Multiple conversions happen simultaneously
   - More efficient for independent calculations
   - Shows different architectural approaches

5. **Data Structure Management**:
   - Dictionary-based data flow
   - Preserves original values alongside conversions
   - Structured output for downstream processing

6. **Batch Processing**:
   - Efficiently processes multiple temperatures
   - Maintains consistent transformation logic
   - Essential for processing large datasets

7. **Medical Application**:
   - Body temperature analysis with clinical context
   - Fever detection and categorization
   - Demonstrates real-world healthcare application

8. **Composition Patterns**:
   - Shows how to combine different types of operations
   - Parallel and sequential processing in the same pipeline
   - Scalable architecture for complex medical workflows

In [ ]:
# Method 1: Sequential conversion (Kelvin → Celsius → Fahrenheit)
sequential_conversion = (
    RunnableLambda(lambda x: x)  # Pass through the input
    | {
        "kelvin": RunnableLambda(lambda x: x),  # Keep original value
        "celsius": kelvin_to_celsius_runnable,  # Convert to Celsius
    }
    | RunnableLambda(
        lambda x: {
            "kelvin": x["kelvin"],
            "celsius": x["celsius"],
            "fahrenheit": celsius_to_fahrenheit(
                x["celsius"]
            ),  # Convert Celsius to Fahrenheit
        }
    )
    | format_result_runnable
)


# Method 2: Direct conversion functions
def kelvin_to_fahrenheit_direct(kelvin_temp):
    """Direct conversion from Kelvin to Fahrenheit"""
    celsius = kelvin_temp - 273.15
    fahrenheit = (celsius * 9 / 5) + 32
    return fahrenheit


# Create parallel conversion chain
parallel_conversion = (
    RunnableLambda(lambda x: x)
    | {
        "kelvin": RunnableLambda(lambda x: x),
        "celsius": kelvin_to_celsius_runnable,
        "fahrenheit": RunnableLambda(kelvin_to_fahrenheit_direct),
    }
    | format_result_runnable
)

In [ ]:
# Test with various temperatures
test_temperatures = [
    273.15,
    300,
    310.15,
    373.15,
]  # 0°C, ~27°C, ~37°C (body temp), 100°C

print("=== Sequential Conversion Results ===")
for temp in test_temperatures:
    result = sequential_conversion.invoke(temp)
    print(f"Sequential: {result['formatted_output']}")

print("\n=== Parallel Conversion Results ===")
for temp in test_temperatures:
    result = parallel_conversion.invoke(temp)
    print(f"Parallel: {result['formatted_output']}")

# Batch processing example
print("\n=== Batch Processing ===")
batch_results = parallel_conversion.batch(test_temperatures)
for result in batch_results:
    print(f"Batch: {result['formatted_output']}")

# Medical context example: Converting body temperature readings
print("\n=== Medical Context: Body Temperature Analysis ===")
body_temps_kelvin = [
    309.15,
    310.15,
    311.15,
    312.15,
]  # Range of body temperatures in Kelvin

medical_analysis = RunnableLambda(lambda x: x) | {
    "temperature_data": parallel_conversion,
    "medical_assessment": RunnableLambda(
        lambda temp_k: {
            "normal_range": 36.1 <= (temp_k - 273.15) <= 37.2,
            "fever_status": (
                "Fever"
                if (temp_k - 273.15) > 37.2
                else "Normal" if (temp_k - 273.15) >= 36.1 else "Hypothermia"
            ),
            "celsius_value": round(temp_k - 273.15, 1),
        }
    ),
}

=== Sequential Conversion Results ===
Sequential: 273.15K = 0.0°C = 32.0°F
Sequential: 300K = 26.85°C = 80.33°F
Sequential: 310.15K = 37.0°C = 98.6°F
Sequential: 373.15K = 100.0°C = 212.0°F

=== Parallel Conversion Results ===
Parallel: 273.15K = 0.0°C = 32.0°F
Parallel: 300K = 26.85°C = 80.33°F
Parallel: 310.15K = 37.0°C = 98.6°F
Parallel: 373.15K = 100.0°C = 212.0°F

=== Batch Processing ===
Batch: 273.15K = 0.0°C = 32.0°F
Batch: 300K = 26.85°C = 80.33°F
Batch: 310.15K = 37.0°C = 98.6°F
Batch: 373.15K = 100.0°C = 212.0°F

=== Medical Context: Body Temperature Analysis ===


In [ ]:
for temp_k in body_temps_kelvin:
    analysis = medical_analysis.invoke(temp_k)
    temp_data = analysis["temperature_data"]
    medical_data = analysis["medical_assessment"]

    print(f"Temperature: {temp_data['formatted_output']}")
    print(f"  Medical Status: {medical_data['fever_status']}")
    print(f"  Normal Range: {medical_data['normal_range']}")
    print()

Temperature: 309.15K = 36.0°C = 96.8°F
  Medical Status: Hypothermia
  Normal Range: False

Temperature: 310.15K = 37.0°C = 98.6°F
  Medical Status: Normal
  Normal Range: True

Temperature: 311.15K = 38.0°C = 100.4°F
  Medical Status: Fever
  Normal Range: False

Temperature: 312.15K = 39.0°C = 102.2°F
  Medical Status: Fever
  Normal Range: False

